## Groundedness Evaluations

In many ways, feedbacks can be thought of as LLM apps themselves. Given text, they return some result. Thinking in this way, we can use TruLens to evaluate and track our feedback quality. We can even do this for different models (e.g. gpt-3.5 and gpt-4) or prompting schemes (such as chain-of-thought reasoning).

This notebook follows an evaluation of a set of test cases generated from human annotated datasets. In particular, we generate test cases from [SummEval](https://arxiv.org/abs/2007.12626) 

In [ ]:
# Import groundedness feedback function
from trulens_eval.feedback import GroundTruthAgreement, Groundedness
from trulens_eval import TruBasicApp, Feedback, Tru, Select, feedback, TruCustomApp
from test_cases import generate_summeval_groundedness_golden_set


Tru().reset_database()

# generator for groundedness golden set
test_cases_gen = generate_summeval_groundedness_golden_set("./datasets/summeval_test_100.json")

# generate x number of test cases
groundedness_golden_set = []
for i in range(50):
    groundedness_golden_set.append(next(test_cases_gen))

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["HUGGINGFACE_API_KEY"] = "..."

### Benchmarking various Groundedness feedback function providers (OpenAI vs Hugginface)

In [ ]:
from trulens_eval.feedback.provider.hugs import Huggingface
import numpy as np

huggingface_provider = Huggingface()
groundedness_hug = Groundedness(groundedness_provider=huggingface_provider)
f_groundedness_hug = Feedback(groundedness_hug.groundedness_measure, name = "Groundedness Huggingface").on_input().on_output().aggregate(groundedness_hug.grounded_statements_aggregator)
def wrapped_groundedness_hug(input, output):
    return np.mean(list(f_groundedness_hug(input, output)[0].values()))
     
    
    
groundedness_openai = Groundedness()
f_groundedness_openai = Feedback(groundedness_openai.groundedness_measure, name = "Groundedness OpenAI").on_input().on_output().aggregate(groundedness_openai.grounded_statements_aggregator)
def wrapped_groundedness_openai(input, output):
    return f_groundedness_openai(input, output)[0]['full_doc_score']

In [ ]:
# Create a Feedback object using the numeric_difference method of the ground_truth object
ground_truth = GroundTruthAgreement(groundedness_golden_set)
# Call the numeric_difference method with app and record
f_groundtruth = Feedback(ground_truth.numeric_difference, name = "Groundedness Smoke Test").on(Select.Record.calls[0].args.args[0]).on(Select.Record.calls[0].args.args[1]).on_output()

In [ ]:
tru_wrapped_groundedness_hug = TruBasicApp(wrapped_groundedness_hug, app_id = "groundedness huggingface", feedbacks=[f_groundtruth])
tru_wrapped_groundedness_openai = TruBasicApp(wrapped_groundedness_openai, app_id = "groundedness openai", feedbacks=[f_groundtruth])


In [ ]:


for i in range(len(groundedness_golden_set[:10])):
    source = groundedness_golden_set[i]["query"]
    response = groundedness_golden_set[i]["response"]
    with tru_wrapped_groundedness_hug as recording:
        tru_wrapped_groundedness_hug.app(source, response)
    with tru_wrapped_groundedness_openai as recording:
        tru_wrapped_groundedness_openai.app(source, response)
    

In [57]:
Tru().get_leaderboard(app_ids=[])

,Groundedness Smoke Test,latency,total_cost
app_id,,,
groundedness huggingface,0.810252,54.571429,0.001080
groundedness openai,0.473333,42.157895,0.001863
